In [1]:
# coding: utf-8
# MultiPerceptron
# queueを使った学習

import os
import time
import pandas as pd
import tensorflow as tf
import threading
from sklearn.utils import shuffle

tf.reset_default_graph()

MODEL_DIR = "../model_car_lidar_queue"
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

data_cols = 3 # センサーの数。left45,front,right45
n_classes = 4 # 予測結果の数。stop,left,forward,right
batch_size = 100 # バッチサイズは10〜100前後に
chunk_size = 100 # FIFOQueueのcapacity

max_step = 100000 # ステップ数

TRAIN_START_INDEX=0
TRAIN_END_INDEX=batch_size
TRAIN_INDEX_I=0
# 学習データを読み込み pandasで読み込みnumpy.ndarrayに変換する
csv_file="../TrainData/car_sensor_train_data.csv"
train_dataset = pd.io.parsers.read_csv(csv_file, header = 0, float_precision = "high").values
TRAIN_NUM = len(train_dataset) # 学習データの行数

def next_train_data(batch_size):
    global TRAIN_START_INDEX
    global TRAIN_END_INDEX
    global TRAIN_NUM
    global TRAIN_INDEX_I
    global train_dataset

    TRAIN_START_INDEX=TRAIN_INDEX_I*batch_size
    TRAIN_END_INDEX=TRAIN_START_INDEX + batch_size
    if TRAIN_NUM < TRAIN_START_INDEX:
        TRAIN_START_INDEX=0
        TRAIN_END_INDEX=TRAIN_START_INDEX + batch_size
        TRAIN_INDEX_I=0
    if TRAIN_NUM < TRAIN_END_INDEX:
        TRAIN_END_INDEX=TRAIN_NUM

    TRAIN_INDEX_I+=1

    if TRAIN_START_INDEX==0:
        # 先頭に戻った時は学習データをシャッフルする
        train_dataset = shuffle(train_dataset, random_state=42) # shuffle train data

    batch_data = train_dataset[TRAIN_START_INDEX:TRAIN_END_INDEX,0:data_cols]
    batch_target = train_dataset[TRAIN_START_INDEX:TRAIN_END_INDEX,data_cols:]
    return batch_data, batch_target


TEST_START_INDEX=0
TEST_END_INDEX=batch_size
TEST_INDEX_I=0
# csv読み込み pandasで読み込みnumpy.ndarrayに変換する
csv_file="../TrainData/car_sensor_test_data.csv"
test_dataset = pd.io.parsers.read_csv(csv_file, header = 0, float_precision = "high").values
TEST_NUM = len(test_dataset) # テストデータの行数

def next_test_data(batch_size):
    global TEST_START_INDEX
    global TEST_END_INDEX
    global TEST_NUM
    global TEST_INDEX_I
    global test_dataset

    TEST_START_INDEX=TEST_INDEX_I*batch_size
    TEST_END_INDEX=TEST_START_INDEX + batch_size
    if TEST_NUM < TEST_START_INDEX:
        TEST_START_INDEX=0
        TEST_END_INDEX=TEST_START_INDEX + batch_size
        TEST_INDEX_I=0
    if TEST_NUM < TEST_END_INDEX:
        TEST_END_INDEX=TEST_NUM

    TEST_INDEX_I+=1

    # テストデータは精度検証用なのでシャッフルは不要
    batch_data = test_dataset[TEST_START_INDEX:TEST_END_INDEX,0:data_cols]
    batch_target = test_dataset[TEST_START_INDEX:TEST_END_INDEX,data_cols:]
    return batch_data, batch_target


def load_and_enqueue(sess):
    while True:
        try:
            batch_data, batch_target = next_train_data(batch_size)
            sess.run(enqueue_op, feed_dict={placeholder_input_data:batch_data, placeholder_input_target:batch_target})
        except tf.errors.CancelledError as e:
            break
    print("finished enqueueing")

with tf.variable_scope("input"):
    placeholder_input_data = tf.placeholder('float', [None, data_cols], name='input_data') # for load_and_enqueue. use dequeue_data_op for prediction
    placeholder_input_target = tf.placeholder('float', name='input_target') # for load_and_enqueue. use dequeue_target_op for prediction
    placeholder_batch_size = tf.placeholder(tf.int32, name='batch_size') # need feed_dict in training sess.run(). don't need for prediction.

with tf.variable_scope("queue"):
    queue = tf.FIFOQueue(
        capacity=chunk_size, # enqueue size
        dtypes=['float', 'float'],
        shapes=[[data_cols], [n_classes]],
        name='FIFOQueue'
    )

    # Enqueue and dequeue operations
    enqueue_op = queue.enqueue_many([placeholder_input_data, placeholder_input_target], name='enqueue_op')
    dequeue_data_op, dequeue_target_op = queue.dequeue_many(placeholder_batch_size, name='dequeue_op') # instead of data/target placeholder


with tf.variable_scope('neural_network_model'):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([data_cols, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(dequeue_data_op,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    # 予測結果
    prediction = tf.add(tf.matmul(l3,output_layer['weights']), output_layer['biases'], name='output_y')

with tf.variable_scope('loss'):
    losses = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=dequeue_target_op)
    loss_op = tf.reduce_mean(losses, name='cost')
    tf.summary.scalar('loss', loss_op)

with tf.variable_scope('accuracy'):
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(dequeue_target_op, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name='accuracy')
    accuracy = tf.Print(accuracy, data=[accuracy], message="accuracy:")
    tf.summary.scalar('accuracy', accuracy)

summary_op = tf.summary.merge_all()

train_op = tf.train.AdamOptimizer(0.0001).minimize(loss_op, name='train_op')
num_batchs = int(TRAIN_NUM/batch_size)

saver = tf.train.Saver()
test_data, test_target =next_test_data(TEST_NUM)
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(MODEL_DIR)
    if ckpt:
        # checkpointファイルから最後に保存したモデルへのパスを取得する
        last_model = ckpt.model_checkpoint_path
        print(("load {0}".format(last_model)))
        # 学習済みモデルを読み込む
        saver.restore(sess, last_model)
        LOAD_MODEL = True
    else:
        print("initialization")
        # 初期化処理
        init_op = tf.global_variables_initializer()
        sess.run(init_op)

    writer = tf.summary.FileWriter('./log', sess.graph)        
    start_time, start_clock = time.time(), time.clock()

    # Start a thread to enqueue data asynchronously, and hide I/O latency.
    coord = tf.train.Coordinator()
    enqueue_thread = threading.Thread(target=load_and_enqueue, args=[sess])
    enqueue_thread.isDaemon()
    enqueue_thread.start()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    try:
        # check the accuracy before training (without feed_dict!)
        print(sess.run(accuracy, feed_dict={placeholder_batch_size:chunk_size})) # check batch_size's data
        for step in range(max_step):
            batch_loss=0
            w_summary=None
            for batch_step in range(num_batchs): # 分割したミニバッチを全て実行する
                _, loss, w_summary = sess.run([train_op, loss_op, summary_op],
                                   feed_dict={placeholder_batch_size:batch_size})
                batch_loss += loss
            if not w_summary is None:
                writer.add_summary(w_summary, step)
    
            ac = sess.run(accuracy, feed_dict={placeholder_batch_size:chunk_size}) # check batch_size's data

            # 全テストデータでの精度を確認する
            full_test_accuracy = accuracy.eval({'queue/dequeue_op:0':test_data,
                                                'queue/dequeue_op:1':test_target})
            if step % 100 == 0:
                print("Step:%d batch_step:%d accuracy:%.8f full_accuracy:%.8f loss:%.8f time:%.8f clock:%.14f" % (step,batch_step,ac,full_test_accuracy,batch_loss,time.time()-start_time,time.clock()-start_clock))

        sess.run(queue.close(cancel_pending_enqueues=True))
    except Exception, e:
        # Report exceptions to the coodinator.
        print(e)
        coord.request_stop(e)
    finally:
        coord.request_stop()
        coord.join(threads)

    saver.save(sess, MODEL_DIR + '/model.ckpt')


    # 全学習データでの精度を確認する
    test_data, test_target =next_train_data(TRAIN_NUM)
    print('Accuracy:',accuracy.eval({dequeue_data_op:test_data,
                                     dequeue_target_op:test_target}))



print("end")


initialization
0.28
Step:0 batch_step:32 accuracy:0.54999995 full_accuracy:0.48079959 loss:27217638.37500000 time:0.77539897 clock:0.39419800000000
Step:100 batch_step:32 accuracy:0.73000002 full_accuracy:0.78151608 loss:198372.50195312 time:12.98833394 clock:20.00354800000000
Step:200 batch_step:32 accuracy:0.64000005 full_accuracy:0.81478649 loss:178414.77392578 time:25.16618609 clock:39.61993600000000
Step:300 batch_step:32 accuracy:0.71999997 full_accuracy:0.82942253 loss:130843.15570068 time:37.29630208 clock:59.22087200000000
Step:400 batch_step:32 accuracy:0.81999993 full_accuracy:0.85265905 loss:111127.12133789 time:49.43093610 clock:78.83004000000000
Step:500 batch_step:32 accuracy:0.78000003 full_accuracy:0.87378305 loss:135634.84606934 time:61.53864408 clock:98.39048300000000
Step:600 batch_step:32 accuracy:0.81999999 full_accuracy:0.85816634 loss:115811.69219971 time:73.66902399 clock:117.95598000000000
Step:700 batch_step:32 accuracy:0.78000003 full_accuracy:0.82127464 los

Step:6200 batch_step:32 accuracy:0.94000006 full_accuracy:0.91648400 loss:67812.20520782 time:748.29987502 clock:1204.33484900000008
Step:6300 batch_step:32 accuracy:0.82999992 full_accuracy:0.88623118 loss:155569.20513916 time:760.29787397 clock:1223.62273000000005
Step:6400 batch_step:32 accuracy:0.87000000 full_accuracy:0.85778916 loss:138035.03601074 time:772.31049705 clock:1242.91883200000007
Step:6500 batch_step:32 accuracy:0.90999997 full_accuracy:0.90011275 loss:58827.27789307 time:784.33189988 clock:1262.20249500000000
Step:6600 batch_step:32 accuracy:0.86000001 full_accuracy:0.86608797 loss:99656.32624531 time:796.31812596 clock:1281.49734600000011
Step:6700 batch_step:32 accuracy:0.95000005 full_accuracy:0.91686118 loss:95714.55496216 time:808.32205701 clock:1300.78307799999993
Step:6800 batch_step:32 accuracy:0.94000000 full_accuracy:0.91346627 loss:63727.82961941 time:820.32117701 clock:1320.07980799999996
Step:6900 batch_step:32 accuracy:0.93000007 full_accuracy:0.8962651

Step:12400 batch_step:32 accuracy:0.91999996 full_accuracy:0.91142929 loss:67309.00828552 time:1493.47643399 clock:2401.91812699999991
Step:12500 batch_step:32 accuracy:0.95000005 full_accuracy:0.90207434 loss:46737.39227295 time:1505.51458001 clock:2421.23968299999979
Step:12600 batch_step:32 accuracy:0.91000009 full_accuracy:0.90177256 loss:69300.06466675 time:1517.53881407 clock:2440.57510499999989
Step:12700 batch_step:32 accuracy:0.89999998 full_accuracy:0.89792490 loss:67034.30297852 time:1529.55069995 clock:2459.88502899999958
Step:12800 batch_step:32 accuracy:0.96000004 full_accuracy:0.92425460 loss:27426.65502930 time:1541.57155704 clock:2479.24178099999972
Step:12900 batch_step:32 accuracy:0.92000002 full_accuracy:0.92168957 loss:93064.73149109 time:1553.60129809 clock:2498.56210199999987
Step:13000 batch_step:32 accuracy:0.92000002 full_accuracy:0.91754007 loss:32805.12462616 time:1565.61449695 clock:2517.87128799999982
Step:13100 batch_step:32 accuracy:0.95000005 full_accur

Step:18500 batch_step:32 accuracy:0.93000001 full_accuracy:0.92402828 loss:48083.28616333 time:2227.36967397 clock:3581.42904199999975
Step:18600 batch_step:32 accuracy:0.96999997 full_accuracy:0.91346616 loss:68409.78869629 time:2239.43007994 clock:3600.78170899999986
Step:18700 batch_step:32 accuracy:0.96000004 full_accuracy:0.93134636 loss:34278.26385307 time:2251.45469999 clock:3620.13099899999997
Step:18800 batch_step:32 accuracy:0.96000004 full_accuracy:0.91995430 loss:39173.38616943 time:2263.50131607 clock:3639.49476399999958
Step:18900 batch_step:32 accuracy:0.98000002 full_accuracy:0.91686118 loss:72130.59423065 time:2275.54572105 clock:3658.85641099999975
Step:19000 batch_step:32 accuracy:0.93000001 full_accuracy:0.92055786 loss:25344.69897270 time:2287.57794189 clock:3678.17417299999988
Step:19100 batch_step:32 accuracy:0.96999997 full_accuracy:0.92101055 loss:62099.53900146 time:2299.61958289 clock:3697.51960199999985
Step:19200 batch_step:32 accuracy:0.95000005 full_accur

Step:24600 batch_step:32 accuracy:0.90000010 full_accuracy:0.92825305 loss:60372.86281967 time:2961.99761891 clock:4762.29473299999972
Step:24700 batch_step:32 accuracy:0.94000000 full_accuracy:0.92946017 loss:75327.25369263 time:2974.05062008 clock:4781.65519299999960
Step:24800 batch_step:32 accuracy:0.97000003 full_accuracy:0.92101049 loss:59385.36941528 time:2986.09853697 clock:4801.01183899999978
Step:24900 batch_step:32 accuracy:0.97000003 full_accuracy:0.93519390 loss:28031.35122013 time:2998.15067601 clock:4820.37651899999946
Step:25000 batch_step:32 accuracy:0.96000004 full_accuracy:0.93081814 loss:30075.86380005 time:3010.20012403 clock:4839.73689199999990
Step:25100 batch_step:32 accuracy:0.95000005 full_accuracy:0.93240249 loss:15735.98122001 time:3022.27393103 clock:4859.08478399999967
Step:25200 batch_step:32 accuracy:0.92000008 full_accuracy:0.92417914 loss:21993.03710556 time:3034.32479000 clock:4878.44774899999993
Step:25300 batch_step:32 accuracy:0.95999998 full_accur

...

Step:73800 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:8889.91943097 clock:14283.89599100000123
Step:73900 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:8901.90843892 clock:14303.22278200000073
Step:74000 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:8913.89347911 clock:14322.55142300000080
Step:74100 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:8925.87530589 clock:14341.88943100000142
Step:74200 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:8937.85564303 clock:14361.19223000000056
Step:74300 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:8949.83729410 clock:14380.50150300000132
Step:74400 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:8961.81446409 clock:14399.81213100000059
Step:74500 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0

Step:80100 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:9648.31914306 clock:15501.91676000000007
Step:80200 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:9660.30734301 clock:15521.20204800000101
Step:80300 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:9672.32804108 clock:15540.49512000000141
Step:80400 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:9684.34365988 clock:15559.79163599999993
Step:80500 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:9696.32324910 clock:15579.08213600000090
Step:80600 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:9708.32215500 clock:15598.39121400000113
Step:80700 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:9720.31680608 clock:15617.72052200000144
Step:80800 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0

Step:86300 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:10374.79460096 clock:16685.41448200000013
Step:86400 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:10386.46332407 clock:16704.48164600000018
Step:86500 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:10398.12839508 clock:16723.54738500000167
Step:86600 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:10409.79289007 clock:16742.59773000000132
Step:86700 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:10421.48450494 clock:16761.68067800000063
Step:86800 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:10433.16478992 clock:16780.77751899999930
Step:86900 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:10444.82139993 clock:16799.81424100000004
Step:87000 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318

Step:92500 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11099.40911102 clock:17867.69916300000114
Step:92600 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11111.07794309 clock:17886.74374800000078
Step:92700 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11122.76870108 clock:17905.82169799999974
Step:92800 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11134.44560790 clock:17924.88779300000169
Step:92900 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11146.12955189 clock:17943.97398900000189
Step:93000 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:11158.23397398 clock:17963.04223700000148
Step:93100 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11169.91288495 clock:17982.11375500000213
Step:93200 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318

Step:98700 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:11851.59769511 clock:19065.14990200000102
Step:98800 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11863.69640899 clock:19084.51320200000191
Step:98900 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:11875.72322989 clock:19103.85851800000091
Step:99000 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11887.82749391 clock:19123.21478700000080
Step:99100 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11899.86947107 clock:19142.58920600000056
Step:99200 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312 loss:0.00000000 time:11912.24055004 clock:19161.98696600000039
Step:99300 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209318 loss:0.00000000 time:11924.33620501 clock:19181.38616499999989
Step:99400 batch_step:32 accuracy:1.00000000 full_accuracy:0.95209312